In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import datetime

# 讀取資料

讀取 2022 年台北市交通事故資料，並保留需要的欄位

In [3]:
traffic_detail = pd.read_csv(
    'DATA/111年-臺北市A1及A2類交通事故明細.csv', encoding = 'ANSI', 
    usecols = ['發生年', '發生月', '發生日', '發生時', '發生分', '區序', '肇事地點', '當事人序', 
               '車種', '4天候', '5光線', '6道路類別', '8道路型態', 
               '9事故位置', '10路面狀況1', '10路面狀況2', '10路面狀況3', '11道路障礙1', '11道路障礙2', 
               '12號誌1', '12號誌2', '13車道劃分-分向', '14車道劃分-分道1', '14車道劃分-分道2', 
               '14車道劃分-分道3', '15事故類型及型態', 'X', 'Y'])

本次研究只聚焦在市區道路以及發生時間介於早上 7 點至晚上 19 點所發生的交通事故，此份事故明細紀錄的方式為一場事故裡有幾個當事人就記錄幾筆資料，因此只留下當事人序號為 1 的資料

In [4]:
# 只留下「市區道路」
traffic_detail = traffic_detail[traffic_detail['6道路類別'] == 5]

# 只留下 7-19 的資料
traffic_detail = traffic_detail[(traffic_detail['發生時'] >= 7) & (traffic_detail['發生時'] <= 19)]

# 取 unique 的交通事故
traffic_detail = traffic_detail[traffic_detail['當事人序'] == 1]

依照上述步驟清理完資料後，共有 20088 筆事故資料

In [5]:
len(traffic_detail)

20088

接著將紀錄事故發生地氣候、路況等欄位展開變成虛擬變數

In [7]:
weather = pd.get_dummies(traffic_detail['4天候'])

weather.columns = ['暴雨', '霧或煙', '雨', '陰', '晴']

# weather = weather.drop('晴', axis = 1)

traffic_detail = traffic_detail.join(weather)

In [8]:
light = pd.get_dummies(traffic_detail['5光線'])

light.columns = ['日間自然光線', '晨或暮光', '夜間(或隧道、地下道、涵洞)有照明', '夜間(或隧道、地下道、涵洞)無照明']

# light = light.drop('日間自然光線', axis = 1)

traffic_detail = traffic_detail.join(light)

In [9]:
road_type = pd.get_dummies(traffic_detail['8道路型態'])

road_type.columns = ['道路型態_三岔路', '道路型態_四岔路', '道路型態_多岔路', '道路型態_隧道', '道路型態_地下道', 
                     '道路型態_橋樑', '道路型態_涵洞', '道路型態_高架道路', '道路型態_彎曲路及附近', 
                     '道路型態_坡路', '道路型態_巷弄', '道路型態_直路', '道路型態_其他單路', '道路型態_圓環', 
                     '道路型態_廣場']

# road_type = road_type.drop('道路型態_四岔路', axis = 1)

traffic_detail = traffic_detail.join(road_type)

In [10]:
accident_location = pd.get_dummies(traffic_detail['9事故位置'])

accident_location.columns = ['事故位置_交岔路口內', '事故位置_交岔口附近', '事故位置_機車待轉區', '事故位置_機車停等區', 
                             '事故位置_交通島_含槽化線', '事故位置_迴轉道', '事故位置_快車道', '事故位置_慢車道', 
                             '事故位置_一般車道_未劃分快慢車道', '事故位置_公車專用道', '事故位置_機車專用道', 
                             '事故位置_機車優先道', '事故位置_路肩、路緣', '事故位置_加速車道', '事故位置_直線匝道', 
                             '事故位置_環道匝道', '事故位置_行人穿越道', '事故位置_穿越道附近', '事故位置_人行道']

# accident_location = accident_location.drop('事故位置_交岔路口內', axis = 1)

traffic_detail = traffic_detail.join(accident_location)

In [11]:
pavement_type = pd.get_dummies(traffic_detail['10路面狀況1'])

pavement_type.columns = ['路面鋪裝_柏油', '路面鋪裝_水泥', '路面鋪裝_碎石', '路面鋪裝_其他鋪裝', '無鋪裝']

# pavement_type = pavement_type.drop('路面鋪裝_柏油', axis = 1)

traffic_detail = traffic_detail.join(pavement_type)

In [12]:
road_situation = pd.get_dummies(traffic_detail['10路面狀況2'])

road_situation.columns = ['路面狀態_油滑', '路面狀態_泥濘', '路面狀態_濕潤', '路面狀態_乾燥']

# road_situation = road_situation.drop('路面狀態_乾燥', axis = 1)

traffic_detail = traffic_detail.join(road_situation)

In [13]:
pavement_defects = pd.get_dummies(traffic_detail['10路面狀況3'])

pavement_defects.columns = ['路面缺陷_路面鬆軟', '路面缺陷_突出(高低)不平', '路面缺陷_有坑洞', '無缺陷']

# pavement_defects = pavement_defects.drop('無缺陷', axis = 1)

traffic_detail = traffic_detail.join(pavement_defects)

In [14]:
obstacle = pd.get_dummies(traffic_detail['11道路障礙1'])

obstacle.columns = ['障礙物_道路工程', '障礙物_有堆積物', '障礙物_路上有停車', '障礙物_其他障礙物', '無障礙物']

# obstacle = obstacle.drop('無障礙物', axis = 1)

traffic_detail = traffic_detail.join(obstacle)

In [15]:
sight_line = pd.get_dummies(traffic_detail['11道路障礙2'])

sight_line.columns = ['視距_彎道', '視距_坡道', '視距_建築物', '視距_樹木、農作物', '視距_路上停放車輛', 
                      '視距_其他', '視距_良好']

# sight_line = sight_line.drop('視距_良好', axis = 1)

traffic_detail = traffic_detail.join(sight_line)

In [16]:
traffic_light = pd.get_dummies(traffic_detail['12號誌1'])

traffic_light.columns = ['行車管制號誌', '行車管制號誌(附設行人專用號誌)', '閃光號誌', '無號誌']

# traffic_light = traffic_light.drop('無號誌', axis = 1)

traffic_detail = traffic_detail.join(traffic_light)

In [17]:
traffic_light_action = pd.get_dummies(traffic_detail['12號誌2'])

traffic_light_action.columns = ['號誌正常', '號誌不正常', '號誌無動作', '無號誌_無號誌']

# traffic_light_action = traffic_light_action.drop('無號誌_無號誌', axis = 1)

traffic_detail = traffic_detail.join(traffic_light_action)

In [18]:
lane_dividing = pd.get_dummies(traffic_detail['13車道劃分-分向'])

lane_dividing.columns = ['分向設施_中央分向島_寬式', '分向設施_中央分向島_窄式附柵欄', '分向設施_中央分向島_窄式無柵欄', 
                         '分向設施_雙向禁止超車線_附標記', '分向設施_雙向禁止超車線_附標記', 
                         '分向設施_單向禁止超車線_附標記', '分向設施_單向禁止超車線_無標記', 
                         '分向設施_行車分向線_附標記', '分向設施_行車分向線_無標記', '分向設施_無分向設施']

# lane_dividing = lane_dividing.drop('分向設施_中央分向島_寬式', axis = 1)

traffic_detail = traffic_detail.join(lane_dividing)

In [19]:
lane_separation_facility = pd.get_dummies(traffic_detail['14車道劃分-分道1'])

lane_separation_facility.columns = ['分道設施_禁止變換車道線_附標記', '分道設施_禁止變換車道線_無標記',
                                    '分道設施_車道線_附標記', '分道設施_車道線_無標記', '分道設施_未繪設車道線']

# lane_separation_facility = lane_separation_facility.drop('分道設施_車道線_無標記', axis = 1)

traffic_detail = traffic_detail.join(lane_separation_facility)

In [20]:
between_fast_slow_lanes = pd.get_dummies(traffic_detail['14車道劃分-分道2'])

between_fast_slow_lanes.columns = ['分道設施_寬式快慢車道分隔島', '分道設施_窄式快慢車道分隔島_附柵欄', 
                                   '分道設施_窄式快慢車道分隔島_無柵欄', '分道設施_快慢車道分隔線', 
                                   '分道設施_未繪設快慢車道分隔線']

# between_fast_slow_lanes = between_fast_slow_lanes.drop('分道設施_未繪設快慢車道分隔線', axis = 1)

traffic_detail = traffic_detail.join(between_fast_slow_lanes)

In [21]:
road_edge_line = pd.get_dummies(traffic_detail['14車道劃分-分道3'])

road_edge_line.columns = ['分道設施_有路面邊線', '分道設施_無路面邊線']

# road_edge_line = road_edge_line.drop('分道設施_無路面邊線', axis = 1)

traffic_detail = traffic_detail.join(road_edge_line)

In [22]:
accident_type = pd.get_dummies(traffic_detail['15事故類型及型態'])

accident_type.columns = ['事故類型_人與汽(機)車_對向通行中', '事故類型_人與汽(機)車_同向通行中', 
                         '事故類型_人與汽(機)車_穿越道路中', '事故類型_人與汽(機)車_在路上嬉戲', 
                         '事故類型_人與汽(機)車_在路上作業中', '事故類型_人與汽(機)車_衝進路中',
                         '事故類型_人與汽(機)車_從停車後(或中)穿出', '事故類型_人與汽(機)車_佇立路邊(外)', 
                         '事故類型_人與汽(機)車_其他', '事故類型_車與車_對撞', '事故類型_車與車_對向擦撞', 
                         '事故類型_車與車_同向擦撞', '事故類型_車與車_追撞', '事故類型_車與車_倒車撞', 
                         '事故類型_車與車_路口交岔撞', '事故類型_車與車_側撞', '事故類型_車與車_其他', 
                         '事故類型_汽(機)車本身_路上翻車', '事故類型_汽(機)車本身_衝出路外', 
                         '事故類型_汽(機)車本身_撞護欄', '事故類型_汽(機)車本身_撞號誌', 
                         '事故類型_汽(機)車本身_撞交通島', '事故類型_汽(機)車本身_撞非固定設施', 
                         '事故類型_汽(機)車本身_撞橋梁、建築物', '事故類型_汽(機)車本身_撞路樹、電桿', 
                         '事故類型_汽(機)車本身_撞動物', '事故類型_汽(機)車本身_撞工程施工', 
                         '事故類型_汽(機)車本身_其他']

# accident_type = accident_type.drop('事故類型_車與車_其他', axis = 1)

traffic_detail = traffic_detail.join(accident_type)

In [23]:
traffic_detail.to_csv('111_taipei_accidence_unique_with_dummies.csv', index = False)

In [25]:
traffic_detail

,發生年,發生月,發生日,發生時,發生分,區序,肇事地點,當事人序,車種,4天候,...,汽(機)車本身_衝出路外,汽(機)車本身_撞護欄,汽(機)車本身_撞號誌,汽(機)車本身_撞交通島,汽(機)車本身_撞非固定設施,汽(機)車本身_撞橋梁、建築物,汽(機)車本身_撞路樹、電桿,汽(機)車本身_撞動物,汽(機)車本身_撞工程施工,汽(機)車本身_其他
25,111,1,1,7,50,08士林區,士林區仰德大道2段與仰德大道2段148巷口,1,C03,8,...,0,0,0,0,0,0,0,0,0,0
27,111,1,1,8,30,03中山區,中山區中山北路2段與錦西街口,1,B01,8,...,0,0,0,0,0,0,0,0,0,0
29,111,1,1,8,55,07信義區,信義區忠孝東路5段與永吉路30巷口,1,C03,7,...,0,0,0,0,0,0,0,0,0,0
31,111,1,1,9,42,01大同區,大同區民權西路106號,1,B03,8,...,0,0,0,0,0,0,0,0,0,0
33,111,1,1,10,0,08士林區,士林區重慶北路4段49巷與重慶北路4段口,1,B03,7,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58169,111,12,31,18,19,07信義區,信義區市民大道6段與松隆路329巷口,1,B01,6,...,0,0,0,0,0,0,0,0,0,0
58171,111,12,31,18,30,12內湖區,內湖區內湖路3段59巷與內湖路3段口,1,B12,6,...,0,0,0,0,0,0,0,0,0,0
58173,111,12,31,19,1,07信義區,信義區吳興街269巷1弄,1,C03,6,...,0,0,0,0,0,0,0,0,0,0
58175,111,12,31,19,15,09北投區,北投區承德路7段92停車格,1,C03,8,...,0,0,0,0,0,0,0,0,0,0


# 下一步

接著，我將下載台北市路口的車流量資料。請參考
[下載車流量資料](./Step2_traffic_transform.ipynb)